In [10]:
import requests
from bs4 import BeautifulSoup
urllist = []
for i in range(1,11,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i)
    urllist.append(url)
headlines = []
for j in urllist:
    data = requests.get(j)
    soup = BeautifulSoup(data.content,"html.parser")
    for i in soup.find_all("h3",class_="entry-title td-module-title"):
        headlines.append(i.find("a")["title"])

In [15]:
urllist

['https://www.opindia.com/latest-news/page/1',
 'https://www.opindia.com/latest-news/page/2',
 'https://www.opindia.com/latest-news/page/3',
 'https://www.opindia.com/latest-news/page/4',
 'https://www.opindia.com/latest-news/page/5',
 'https://www.opindia.com/latest-news/page/6',
 'https://www.opindia.com/latest-news/page/7',
 'https://www.opindia.com/latest-news/page/8',
 'https://www.opindia.com/latest-news/page/9',
 'https://www.opindia.com/latest-news/page/10']

In [16]:
data

<Response [200]>

In [17]:
soup

<!DOCTYPE html >

<html lang="en-GB">
<head>
<meta charset="utf-8"/>
<title>Today's Latest English News Reports from India | Archives of OpIndia - Page 10</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<script src="/cdn-cgi/apps/head/PF-ojECnpW2hPKRp07hY76hd0Qs.js"></script><link href="https://www.opindia.com/xmlrpc.php" rel="pingback"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<link href="https://www.opindia.com/latest-news/page/10/" rel="canonical"/><link href="https://www.opindia.com/latest-news/page/9/" rel="prev"/><link href="https://www.opindia.com/latest-news/page/11/" rel="next"/><link href="https://www.opindia.com/wp-content/uploads/2018/10/opindia-logo-300x300.png" rel="icon" type="image/png"/>
<meta content="Latest Breaking News and Opinions from OpIndia.com. List of all news reports and opinions published by OpIndia in a chronological order." name="description"/>
<meta conten

In [14]:
headlines

['ED raids properties linked with Dawood Ibrahim, also to probe connection with a senior politician in Mumbai',
 'Hijab row: College in Madhya Pradesh issues order asking students not to wear any religion-specific clothing inside campus',
 'Lalu Yadav found guilty in 5th Fodder Scam case: Illegally collected Rs 139.5 crores from the Doranda treasury',
 'Karnataka burqa row: 13 Muslim girls boycott preparatory exams, say will quit school if not allowed to wear hijab to classes',
 'Gujarat: Kundan Kothiya becomes sixth AAP councillor of Surat to join BJP, cites ‘harassment and insult’ in party',
 'Pulwama terror attack: The ominous portents that mainstreaming of ‘gau mutra’ jibes hold',
 'Foreign funding, radical Islamic organisations and hijab protests: A plan to mobilise Muslim youth against India',
 'Kishan and Lavanya get forgotten while burqa-clad Sharia loyalists get echoed in Supreme Court: A tale of two Indias',
 'Ajit Anjum apologises for “chor chamar” slur: Read how British ins

In [18]:
articles = []

def article_fetch(url):
    data1 = requests.get(url)
    soup1 = BeautifulSoup(data1.content,"html.parser")
    art = ""
    for i in soup1.find_all("div",class_="tdb-block-inner td-fix-index"):
        art = art+ str(i.text)
    return(art)    

In [21]:
for i in urllist:
    article = article_fetch(i)
    articles.append(article[27:len(article)])

In [22]:
art = []
for i in range(0,len(articles)):
    art.append(articles[i][27:len(articles[i])])

In [23]:
dates = []
for i in articles:
    dates.append(i[0:20])

In [24]:
import pandas as pd
Q = pd.DataFrame([dates,headlines,art]).T
Q.columns = ["Date","Headline","News_article"]
Q.head()

,Date,Headline,News_article
0,,ED raids properties linked with Dawood Ibrahim...,
1,,Hijab row: College in Madhya Pradesh issues or...,
2,,Lalu Yadav found guilty in 5th Fodder Scam cas...,
3,,Karnataka burqa row: 13 Muslim girls boycott p...,
4,,Gujarat: Kundan Kothiya becomes sixth AAP coun...,


In [25]:
import re
ppd = []
for i in art:
    q = i.lower()
    q = re.sub("[^a-zA-Z ]","",q)
    ppd.append(q)

# Preprocessing

In [ ]:
from nltk.stem import LancasterStemmer
ps = LancasterStemmer()

from nltk.stem import WordNetLemmatizer 
lem = WordNetLemmatizer()
E = []
ppdf = []
for j in range(0,len(ppd)):
    wnl = []
    for i in ppd[j].split(" "):
        wnl.append(lem.lemmatize(ps.stem(i)))
        E.append(lem.lemmatize(ps.stem(i)))
        
    ppdf.append(wnl)

In [29]:
len(ppdf)

0

In [28]:
Q.head()

,Date,Headline,News_article
0,,ED raids properties linked with Dawood Ibrahim...,
1,,Hijab row: College in Madhya Pradesh issues or...,
2,,Lalu Yadav found guilty in 5th Fodder Scam cas...,
3,,Karnataka burqa row: 13 Muslim girls boycott p...,
4,,Gujarat: Kundan Kothiya becomes sixth AAP coun...,


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(E).toarray()

In [ ]:
words = tfidf.get_feature_names()

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
model = km.fit(X)

In [ ]:
model.labels_

In [ ]:
Q1 = pd.DataFrame([words,model.labels_]).T

In [33]:
P = {}

In [ ]:
for i in Q1.index:
    P[i]=E[i]

In [ ]:
list(enumerate(model.cluster_centers_.argsort()))

In [ ]:
for i,j in list(enumerate(model.cluster_centers_.argsort())):
    for w in j:
        print(i,P[w])

In [ ]:
for i,j in list(enumerate(model.cluster_centers_.argsort()[:,:-30:-1])):
    for w in j:
        print(i,P[w])

In [ ]:
from sklearn.decomposition import NMF
nm = NMF(n_components=3,random_state=21)
model = nm.fit(X)

In [ ]:
inp = ["Republic TV sued for its content"]

q = nm.transform(tfidf.transform(inp)).argsort()[0][2]

In [ ]:
print("Following article belongs to cluster: ",q)